##init

In [0]:
import pyspark.sql.functions as F


##Read from Silver layer Table

In [0]:
%sql
SELECT * FROM airbnb.silver.listing_table;

##Bisiness Transormation and Aggregation

In [0]:
%sql
--  Business rules (listing price > 0 and minimum_nights >= 1)
--creating a new table neighbourhood_insights 
CREATE OR REPLACE TABLE airbnb.gold.neighbourhood_insights AS
SELECT
    neighbourhood,
    ROUND(AVG(price), 2) AS avg_price,
    ROUND(MAX(price), 2) AS max_price,
    ROUND(MIN(price), 2) AS min_price,
    COUNT(id) AS total_listings
FROM airbnb.silver.listing_table
WHERE price > 0 AND minimum_nights >= 1
GROUP BY neighbourhood;

In [0]:
%sql
-- Business rules (listing price > 0 and minimum_nights >= 1)
--creating a new table Listing_details

CREATE OR REPLACE TABLE airbnb.gold.listing_details AS
SELECT
    id AS listing_id,
    price,
    number_of_reviews,
    minimum_nights,
    neighbourhood,
    host_id,
    room_type
FROM airbnb.silver.listing_table
WHERE price > 0 AND minimum_nights >= 1;

In [0]:
%sql
-- Business rules (listing price > 0 and minimum_nights >= 1)
--creating a new table host_ranking

CREATE OR REPLACE TABLE airbnb.gold.host_ranking AS
WITH host_metrics AS (
    SELECT
        host_id,
        COUNT(id) AS total_listings,
        SUM(number_of_reviews) AS total_reviews,
        ROUND(AVG(price), 2) AS avg_price,
        MAX(last_review) AS last_review_date
    FROM airbnb.silver.listing_table
    WHERE price > 0 AND minimum_nights >= 1
    GROUP BY host_id
)
SELECT
    host_id,
    total_listings,
    total_reviews,
    avg_price,
    CASE
        WHEN last_review_date IS NOT NULL 
             AND last_review_date != '-' 
             AND last_review_date >= DATEADD(day, -30, CURRENT_DATE) 
        THEN 'Active'
        ELSE 'Inactive'
    END AS activity_status,
    ROW_NUMBER() OVER (ORDER BY total_listings DESC) AS rank_by_listings,
    ROW_NUMBER() OVER (ORDER BY total_reviews DESC) AS rank_by_reviews
FROM host_metrics;